### Performing tasks according to the business rules

In [98]:
import pandas as pd
import numpy as np
import sqlite3
import csv
import json
from sqlalchemy import create_engine

In [81]:
# Reading the data and converting it to dataFrame
dfa = pd.read_csv(r"C:\Users\rrak\Downloads\order_region_a.csv")
dfb = pd.read_csv(r"C:\Users\rrak\Downloads\order_region_b.csv")

In [82]:
# 1. Combine the data from both regions into a single dataframe.
combined_data = pd.concat([dfa, dfb])

In [22]:
combined_data.columns

Index(['OrderId', 'OrderItemId', 'QuantityOrdered', 'ItemPrice',
       'PromotionDiscount', 'batch_id'],
      dtype='object')

In [83]:
# 2. Add a column total_sales which is calculated as QuantityOrdered * ItemPrice.
combined_data["total_sales"] = combined_data["QuantityOrdered"] * combined_data["ItemPrice"]

In [85]:
# 3. Add a column region to identify the region of the sales record (A or B).
midpoint = (len(combined_data) // 2) + 1
combined_data["region"] = ['A'] * midpoint + ['B'] *(len(combined_data) - midpoint)

In [86]:
def extract_amount(value):
    if isinstance(value, str):  # Convert JSON string to dict
        try:
            value = json.loads(value)
        except json.JSONDecodeError:
            return 0  # Default value in case of error
    return float(value.get("Amount", 0))  # Extract "Amount"

combined_data['DiscountAmount'] = combined_data['PromotionDiscount'].apply(extract_amount)

In [101]:
# 4. Dropping dupliactes
combined_data = combined_data.drop_duplicates(subset=['OrderId'])

In [102]:
# 5. Add a new column net_sale, calculated as total_sales - PromotionDiscount.
combined_data['net_sale'] = combined_data['total_sales'] - combined_data['DiscountAmount']

C:\Users\rrak\AppData\Local\Temp\ipykernel_40036\397335015.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_data['net_sale'] = combined_data['total_sales'] - combined_data['DiscountAmount']


In [103]:
# 6. Exclude orders where the total sales amount is negative or zero after applying discounts.
net_sales_above_zero = combined_data.loc[combined_data["total_sales"] <= 0]

In [104]:
# Dropping the extra column
data = net_sales_above_zero.drop(columns=['DiscountAmount'])

In [105]:
engine = create_engine("sqlite:///combined_region_database.db")

In [106]:
data.to_sql("combined_region_database", con=engine, if_exists="replace", index=False)

55

### SQL Queries

In [ ]:
# 1. Count the total number of records.
select count(OrderId) 
from combined_region_database

In [ ]:
# 2. Find the total sales amount by region.
select sum(net_sales)
from combined_region_database
group by region

In [ ]:
# 3. Find the average sales amount per transaction.
select avg(net_sales) 
from combined_region_database

In [ ]:
# 4. Ensure there are no duplicate OrderId values.
ALTER TABLE combined_region_database ADD CONSTRAINT unique_order UNIQUE (OrderId);

In [ ]:
DELETE FROM combined_region_database
WHERE OrderId IN (
    SELECT rowid FROM (
        SELECT rowid, 
               ROW_NUMBER() OVER (PARTITION BY OrderId ORDER BY OrderId DESC) AS row_num
        FROM sales_data
    ) t
    WHERE row_num > 1
);

